## Datos ICFES


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels
import sklearn.linear_model as lm
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from statsmodels.stats.outliers_influence import variance_inflation_factor
#Combinar vab de calidad de vida. PCA categórico??

In [9]:
#Datos eliminados. Archivo muy pesado. Se trabaja con el archivo limpiado data.csv
data=pd.read_csv("C:/Users/marti/OneDrive/Documentos/personal/Universidad/Econometria/Saber_11.csv")
#Limpieza
column_descartadas=data.columns[[0,1,3,4,5,6,8,9,10,11,12,13,16,19,20,22,24,25,26,27,28,33,37,38,41,43,44,45,46,47,48,51,52,53,55,56,57,58,59,61,62,64,65,67,68,70,71,73,74,76,77,78,79,80,81]]#Lista de nombres de columnas a descartar"ESTU_TIPODOCUMENTO","ESTU_NACIONALIDAD","ESTU_NACIONALIDAD","ESTU_"
data.drop(column_descartadas, inplace=True,axis=1) 

#Eliminación filas sin info:
data=data[data.ESTU_GENERO != "-"]#eliminación filas sin género
data=data[(data.FAMI_ESTRATOVIVIENDA != "-" ) & (data.FAMI_ESTRATOVIVIENDA !="Sin Estrato")& ~( data.FAMI_ESTRATOVIVIENDA.isna())] #Eliminación filas sin especificación de estrato (& realiza elemento a elemento)
data=data[data.FAMI_PERSONASHOGAR != "-"]
data=data[(data.FAMI_EDUCACIONPADRE != "-")&(data.FAMI_EDUCACIONPADRE != "No Aplica")&(data.FAMI_EDUCACIONPADRE !="No sabe")]
data=data[(data.FAMI_EDUCACIONMADRE != "-")&(data.FAMI_EDUCACIONMADRE != "No Aplica")&(data.FAMI_EDUCACIONMADRE !="No sabe")]

data=data[(data.FAMI_TIENEINTERNET != "-")]
data=data[(data.FAMI_TIENECOMPUTADOR != "-")]


data=data[(data.FAMI_NUMLIBROS != "-")]
data=data[(data.ESTU_DEDICACIONLECTURADIARIA !="-")]
data=data[(data.ESTU_DEDICACIONINTERNET !="-")]

data=data[(data.FAMI_COMELECHEDERIVADOS != "-")]
data=data[(data.FAMI_COMECARNEPESCADOHUEVO != "-")]
data=data[(data.FAMI_COMECEREALFRUTOSLEGUMBRE != "-")]

data=data[(data.ESTU_HORASSEMANATRABAJA != "-")]


#Resumen info
# data.info()
#print(data.describe(include=["O"]))#Descripción categóricas
#data.head()
#Creación variables

#Variable género es útil en regresión individual
#data.ESTU_GENERO=pd.get_dummies(data.ESTU_GENERO).M #0 si es mujer, 1 si es hombre
data.ESTU_TIENEETNIA= (data.ESTU_TIENEETNIA=="Si").astype(int)
data.FAMI_ESTRATOVIVIENDA=data.FAMI_ESTRATOVIVIENDA.str.split(" ").str[1].astype(int) #Extraer solo el número del estrato

condiciones = [
    data['FAMI_PERSONASHOGAR'].eq('1 a 2'), data['FAMI_PERSONASHOGAR'].eq('3 a 4'),
    data['FAMI_PERSONASHOGAR'].eq('5 a 6'),data['FAMI_PERSONASHOGAR'].eq('7 a 8'),
    data['FAMI_PERSONASHOGAR'].eq('9 o más')
]
opciones = [1,3,5,7,9]
data.FAMI_PERSONASHOGAR = np.select(condiciones, opciones, default=0)
#data.FAMI_PERSONASHOGAR =(data.FAMI_PERSONASHOGAR >= 7).astype(int)

#Conectividad
condiciones = [
    data.FAMI_TIENECOMPUTADOR.eq("Si") |data.FAMI_TIENEINTERNET.eq('Si')
]
opciones=[1]
data.FAMI_TIENECOMPUTADOR=np.select(condiciones, opciones, default=0)


data.drop("FAMI_TIENEINTERNET", inplace=True,axis=1)
data.rename(columns = {'FAMI_TIENECOMPUTADOR':'conectividad'}, inplace = True) #se puede cambiar: asignar a data["conectividad"] directamente

#Libros
condiciones = [
    data['FAMI_NUMLIBROS'].eq('0 A 10 LIBROS'), data['FAMI_NUMLIBROS'].eq('11 A 25 LIBROS'),
    data['FAMI_NUMLIBROS'].eq('26 A 100 LIBROS'),data['FAMI_NUMLIBROS'].eq('MÁS DE 100 LIBROS')
]
opciones = [10,25,50,100]
data.FAMI_NUMLIBROS=np.select(condiciones, opciones, default=0)


#Dedicación
condiciones = [
    data['ESTU_DEDICACIONLECTURADIARIA'].eq('No leo por entretenimiento'),
    data['ESTU_DEDICACIONLECTURADIARIA'].eq('30 minutos o menos'),
    data['ESTU_DEDICACIONLECTURADIARIA'].eq('Entre 30 y 60 minutos'),
    data['ESTU_DEDICACIONLECTURADIARIA'].eq('Entre 1 y 2 horas'),
    data['ESTU_DEDICACIONLECTURADIARIA'].eq('Más de 2 horas')
]
opciones = [0,3,6,12,18]
data.ESTU_DEDICACIONLECTURADIARIA=np.select(condiciones, opciones, default=0)


condiciones = [
    data['ESTU_DEDICACIONINTERNET'].eq('No Navega Internet'),
    data['ESTU_DEDICACIONINTERNET'].eq('30 minutos o menos'),
    data['ESTU_DEDICACIONINTERNET'].eq('Entre 30 y 60 minutos'),
    data['ESTU_DEDICACIONINTERNET'].eq('Entre 1 y 3 horas'),
    data['ESTU_DEDICACIONINTERNET'].eq('Más de 3 horas')
]
opciones = [0,3,6,12,18]
data.ESTU_DEDICACIONINTERNET=np.select(condiciones, opciones, default=0)

#Alimentación
condiciones = [
    data['FAMI_COMELECHEDERIVADOS'].eq('3 a 5 veces por semana')| data['FAMI_COMELECHEDERIVADOS'].eq('Todos o casi todos los días')
]
opciones = [1]
data.FAMI_COMELECHEDERIVADOS=np.select(condiciones, opciones, default=0)

condiciones = [
    data['FAMI_COMECARNEPESCADOHUEVO'].eq('3 a 5 veces por semana')| data['FAMI_COMECARNEPESCADOHUEVO'].eq('Todos o casi todos los días')
]
opciones = [1]
data.FAMI_COMECARNEPESCADOHUEVO=np.select(condiciones, opciones, default=0)

condiciones = [
    data['FAMI_COMECEREALFRUTOSLEGUMBRE'].eq('3 a 5 veces por semana')| data['FAMI_COMECEREALFRUTOSLEGUMBRE'].eq('Todos o casi todos los días')
]
opciones = [1]
data.FAMI_COMECEREALFRUTOSLEGUMBRE=np.select(condiciones, opciones, default=0)

data["alimentacion"]=data[['FAMI_COMELECHEDERIVADOS', 'FAMI_COMECARNEPESCADOHUEVO','FAMI_COMECEREALFRUTOSLEGUMBRE']].sum(axis=1)
data.alimentacion=(data.alimentacion>=2).astype(int)

data.drop(['FAMI_COMELECHEDERIVADOS','FAMI_COMECARNEPESCADOHUEVO','FAMI_COMECEREALFRUTOSLEGUMBRE'], inplace=True,axis=1)
#Trabaja
data.ESTU_HORASSEMANATRABAJA=(~ data.ESTU_HORASSEMANATRABAJA.eq('0')).astype(int)

#data_colegios1
# condiciones = [
#     data.FAMI_EDUCACIONPADRE.eq("Secundaria (Bachillerato) completa") |data.FAMI_EDUCACIONPADRE.eq('Educación profesional completa')
#     |data.FAMI_EDUCACIONPADRE.eq("Técnica o tecnológica completa")|data.FAMI_EDUCACIONPADRE.eq("Técnica o tecnológica incompleta")
#     |data.FAMI_EDUCACIONPADRE.eq("Educación profesional incompleta")| data.FAMI_EDUCACIONPADRE.eq("Postgrado")
# ]
# opciones=[1]
# data.FAMI_EDUCACIONPADRE=np.select(condiciones, opciones, default=0)
# condiciones = [
#     data.FAMI_EDUCACIONMADRE.eq("Secundaria (Bachillerato) completa") | data.FAMI_EDUCACIONMADRE.eq('Educación profesional completa')
#     | data.FAMI_EDUCACIONMADRE.eq("Técnica o tecnológica completa")|data.FAMI_EDUCACIONMADRE.eq("Técnica o tecnológica incompleta")
#     |data.FAMI_EDUCACIONMADRE.eq("Educación profesional incompleta")| data.FAMI_EDUCACIONMADRE.eq("Postgrado")
# ]
# opciones=[1]
# data.FAMI_EDUCACIONMADRE=np.select(condiciones, opciones, default=0)

#Data_colegios2
# condiciones = [
#     data.FAMI_EDUCACIONPADRE.eq("Secundaria (Bachillerato) completa") |data.FAMI_EDUCACIONPADRE.eq('Educación profesional completa')
#     |data.FAMI_EDUCACIONPADRE.eq("Técnica o tecnológica completa")|data.FAMI_EDUCACIONPADRE.eq("Técnica o tecnológica incompleta")
#     |data.FAMI_EDUCACIONPADRE.eq("Educación profesional incompleta")| data.FAMI_EDUCACIONPADRE.eq("Postgrado")
#     |data.FAMI_EDUCACIONMADRE.eq("Secundaria (Bachillerato) completa") | data.FAMI_EDUCACIONMADRE.eq('Educación profesional completa')
#     |data.FAMI_EDUCACIONMADRE.eq("Técnica o tecnológica completa")|data.FAMI_EDUCACIONMADRE.eq("Técnica o tecnológica incompleta")
#     |data.FAMI_EDUCACIONMADRE.eq("Educación profesional incompleta")| data.FAMI_EDUCACIONMADRE.eq("Postgrado")
# ]

# opciones=[1]
# data["Educacion_padres"]=np.select(condiciones, opciones, default=0)
# data.drop(["FAMI_EDUCACIONPADRE","FAMI_EDUCACIONMADRE"], inplace=True,axis=1)

#data_colegios3
condiciones = [
    data.FAMI_EDUCACIONPADRE.eq("Ninguno") |data.FAMI_EDUCACIONMADRE.eq('Ninguno'),
    data.FAMI_EDUCACIONPADRE.eq("Primaria incompleta") |data.FAMI_EDUCACIONMADRE.eq('Primaria incompleta'),
    data.FAMI_EDUCACIONPADRE.eq("Primaria completa") |data.FAMI_EDUCACIONMADRE.eq('Primaria completa'),
    data.FAMI_EDUCACIONPADRE.eq("Secundaria (Bachillerato) incompleta") |data.FAMI_EDUCACIONMADRE.eq('Secundaria (Bachillerato) incompleta'),
    data.FAMI_EDUCACIONPADRE.eq("Secundaria (Bachillerato) completa") |data.FAMI_EDUCACIONMADRE.eq('Secundaria (Bachillerato) completa'),
    data.FAMI_EDUCACIONPADRE.eq("Técnica o tecnológica incompleta") |data.FAMI_EDUCACIONMADRE.eq('Técnica o tecnológica incompleta'),
    data.FAMI_EDUCACIONPADRE.eq("Técnica o tecnológica completa") |data.FAMI_EDUCACIONMADRE.eq('Técnica o tecnológica completa'),
    data.FAMI_EDUCACIONPADRE.eq("Educación profesional incompleta") |data.FAMI_EDUCACIONMADRE.eq('Educación profesional incompleta'),
    data.FAMI_EDUCACIONPADRE.eq("Educación profesional completa") |data.FAMI_EDUCACIONMADRE.eq('Educación profesional completa'),
    data.FAMI_EDUCACIONPADRE.eq("Postgrado") |data.FAMI_EDUCACIONMADRE.eq('Postgrado'),
]

opciones=[0,1,2,3,4,5,6,7,8,9]
data["Educacion_padres"]=np.select(condiciones, opciones, default=0)
data.drop(["FAMI_EDUCACIONPADRE","FAMI_EDUCACIONMADRE"], inplace=True,axis=1)

#Oficiales/no oficiales
data.COLE_NATURALEZA=(data.COLE_NATURALEZA.eq("NO OFICIAL")).astype(int)
#Rural/urbano
data.COLE_AREA_UBICACION=(data.COLE_AREA_UBICACION.eq("URBANO")).astype(int)

#Agrupación por colegio para creación de archivo (caso regresión 1)
#'ESTU_DEDICACIONLECTURADIARIA','ESTU_DEDICACIONINTERNET','FAMI_NUMLIBROS' tienen relevancia para análisis individual
#data_colegios1=data.groupby(['COLE_NOMBRE_ESTABLECIMIENTO','COLE_NATURALEZA','COLE_AREA_UBICACION','COLE_DEPTO_UBICACION'],as_index=False)[['ESTU_TIENEETNIA','FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE','ESTU_DEDICACIONLECTURADIARIA','ESTU_DEDICACIONINTERNET','FAMI_NUMLIBROS','conectividad','alimentacion','ESTU_HORASSEMANATRABAJA','PUNT_GLOBAL']].mean()

#data_colegios2 
data_colegios2=data.groupby(['COLE_NOMBRE_ESTABLECIMIENTO','COLE_NATURALEZA','COLE_AREA_UBICACION','COLE_DEPTO_UBICACION'],as_index=False)[['ESTU_TIENEETNIA','FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','Educacion_padres','ESTU_DEDICACIONLECTURADIARIA','ESTU_DEDICACIONINTERNET','FAMI_NUMLIBROS','conectividad','alimentacion','ESTU_HORASSEMANATRABAJA','PUNT_GLOBAL']].mean()

#transformación variables porcentuales al intervalo [0,100]
data_colegios2['ESTU_TIENEETNIA']=100*data_colegios2['ESTU_TIENEETNIA']
#data_colegios2['Educacion_padres']=100*data_colegios2['Educacion_padres']#para regresión 2 punicamente
#data_colegios1['FAMI_EDUCACIONMADRE']=100*data_colegios1['FAMI_EDUCACIONMADRE'] #para regresión 1
data_colegios2['conectividad']=100*data_colegios2['conectividad']
data_colegios2['alimentacion']=100*data_colegios2['alimentacion']
data_colegios2['ESTU_HORASSEMANATRABAJA']=100*data_colegios2['ESTU_HORASSEMANATRABAJA']

#Creación archivo
# data_colegios2.to_csv(path_or_buf="C:/Users/marti/OneDrive/Documentos/personal/Universidad/Econometria/Proyecto/Proyecto-Econometria-I/data_colegios3.csv")


c:\Users\marti\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (76,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#Lectura de archivo creado 
# datos=pd.read_csv("C:/Users/marti/OneDrive/Documentos/personal/Universidad/Econometria/Proyecto/Proyecto-Econometria-I/data_colegios1.csv")
# data_reg=datos[['COLE_NATURALEZA', 'COLE_AREA_UBICACION', 'ESTU_TIENEETNIA', 'FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE','ESTU_DEDICACIONLECTURADIARIA','ESTU_DEDICACIONINTERNET','FAMI_NUMLIBROS','conectividad','alimentacion','ESTU_HORASSEMANATRABAJA','PUNT_GLOBAL']]

# datos=pd.read_csv("C:/Users/marti/OneDrive/Documentos/personal/Universidad/Econometria/Proyecto/Proyecto-Econometria-I/data_colegios2.csv")
# data_reg=datos[['COLE_NATURALEZA', 'COLE_AREA_UBICACION', 'ESTU_TIENEETNIA', 'FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','Educacion_padres','ESTU_DEDICACIONLECTURADIARIA','ESTU_DEDICACIONINTERNET','FAMI_NUMLIBROS','conectividad','alimentacion','ESTU_HORASSEMANATRABAJA','PUNT_GLOBAL']]

datos=pd.read_csv("C:/Users/marti/OneDrive/Documentos/personal/Universidad/Econometria/Proyecto/Proyecto-Econometria-I/data_colegios3.csv")
data_reg3=datos[['COLE_NATURALEZA', 'COLE_AREA_UBICACION', 'ESTU_TIENEETNIA', 'FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','Educacion_padres','ESTU_DEDICACIONLECTURADIARIA','ESTU_DEDICACIONINTERNET','FAMI_NUMLIBROS','conectividad','alimentacion','ESTU_HORASSEMANATRABAJA','PUNT_GLOBAL']]


In [ ]:

#sns.boxplot(x="COLE_AREA_UBICACION",y="PUNT_GLOBAL",data=data_reg)
#sns.boxplot(x="COLE_NATURALEZA",y="PUNT_GLOBAL",data=data_reg)#.iloc[:,1:])

#sns.heatmap(data_reg.corr(),cmap="Blues")
#data_reg.corr()
#data_reg.info()

#sns.FacetGrid(data_reg,hue="COLE_AREA_UBICACION",height=3)\
#   .map(sns.distplot,"PUNT_GLOBAL")\
#    .add_legend() 
#plt.show()


In [ ]:
# cuanti=data_reg[['COLE_NATURALEZA','FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE','conectividad','alimentacion','ESTU_HORASSEMANATRABAJA','PUNT_GLOBAL']]
# plt.close()
# sns.set_style("whitegrid")
# sns.pairplot(cuanti,hue="COLE_NATURALEZA",height=3)
# plt.show()

cuanti=data_reg3[['COLE_NATURALEZA','FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','Educacion_padres','conectividad','alimentacion','ESTU_HORASSEMANATRABAJA','PUNT_GLOBAL']]
plt.close()
sns.set_style("whitegrid")
sns.pairplot(cuanti,hue="COLE_NATURALEZA",height=3)
plt.show()

In [ ]:
# X = sm.add_constant(data_reg[['COLE_NATURALEZA', 'COLE_AREA_UBICACION', 'ESTU_TIENEETNIA', 'FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE','ESTU_DEDICACIONLECTURADIARIA','ESTU_DEDICACIONINTERNET','FAMI_NUMLIBROS','conectividad','alimentacion','ESTU_HORASSEMANATRABAJA']])
# reg = sm.OLS(data_reg["PUNT_GLOBAL"],X)
# model_fit=reg.fit()
# print(model_fit.summary())


X = sm.add_constant(data_reg3[['COLE_NATURALEZA', 'COLE_AREA_UBICACION', 'ESTU_TIENEETNIA', 'FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','Educacion_padres','ESTU_DEDICACIONLECTURADIARIA','ESTU_DEDICACIONINTERNET','FAMI_NUMLIBROS','conectividad','alimentacion','ESTU_HORASSEMANATRABAJA']])
reg = sm.OLS(data_reg3["PUNT_GLOBAL"],X)
model_fit=reg.fit()
print(model_fit.summary())

In [ ]:
# names = ['Lagrange multiplier statistic', 'p-value',
#         'f-value', 'f p-value']
# test = sms.het_breuschpagan(results2.resid, results2.model.exog)

# lzip(names,test)

plt.style.use('seaborn') # pretty matplotlib plots

# fitted values (need a constant term for intercept)
model_fitted_y = model_fit.fittedvalues

model_exog=model_fit.model.exog
# model residuals
model_residuals = model_fit.resid
# normalized residuals
model_norm_residuals = model_fit.get_influence().resid_studentized_internal
# absolute squared normalized residuals
model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))
# absolute residuals
model_abs_resid = np.abs(model_residuals)

In [ ]:
plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)

plot_lm_1.axes[0] = sns.residplot(x = model_fitted_y[:500],
                    y = model_residuals[:500],
                    data = data_reg,
                    lowess = True)
  
# show the plot


plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')
plt.show()

In [ ]:
#Regresión de educación padres en función de las demás vab
X = sm.add_constant(data_reg[['COLE_NATURALEZA', 'COLE_AREA_UBICACION', 'ESTU_TIENEETNIA', 'FAMI_ESTRATOVIVIENDA','FAMI_PERSONASHOGAR','conectividad','alimentacion']])
reg = sm.OLS(data_reg['Educacion_padres'],X)
resultados=reg.fit()
print(resultados.summary())

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

In [ ]:
#Ejemplo reducido 

X = sm.add_constant(data_reg[['FAMI_ESTRATOVIVIENDA','Educacion_padres','alimentacion','ESTU_HORASSEMANATRABAJA']])
reg = sm.OLS(data_reg["PUNT_GLOBAL"],X)
resultados=reg.fit()
print(resultados.summary())

fig = sm.graphics.plot_partregress_grid(resultados)
fig.tight_layout(pad=1.0)

In [ ]:
names = ['Lagrange multiplier statistic', 'p-value',
        'f-value', 'f p-value']
test = sms.het_breuschpagan(model_residuals, model_exog)

lzip(names,test)

In [ ]:
regGLS = sm.GLS(data_reg["PUNT_GLOBAL"],X)
model2_fit=regGLS.fit()
print(model2_fit.summary())

In [ ]:

model2_exog=model2_fit.model.exog
# model residuals
model2_residuals = model2_fit.resid

names = ['Lagrange multiplier statistic', 'p-value',
        'f-value', 'f p-value']
test = sms.het_breuschpagan(model2_residuals, model2_exog)

lzip(names,test)